### Bibliotecas

In [192]:
import numpy as np

### Implementação dos Modelos Lineares de Aprendizagem de Máquina

In [196]:
class PerceptronLA:
    def __init__(self, _X, _Y):
        self.X = np.hstack((np.ones((_X.shape[0], 1)), _X))
        self.Y = _Y
    
    def fit(self):

        self.W = np.zeros(self.X.shape[1])
        indice_class_incorretas = np.arange(len(self.X))

        while (len(indice_class_incorretas) > 0):
            indice_aleatorio = np.random.choice(indice_class_incorretas)

            self.W = self.W + (self.Y[indice_aleatorio] * self.X[indice_aleatorio])

            lista_provisoria = []

            for i in range(len(self.X)):
                classificacao_atual = np.sign(self.W @ self.X[i])

                if classificacao_atual != self.Y[i]:
                    lista_provisoria.append(i)
            
            indice_class_incorretas = np.array(lista_provisoria)
    
    def predict(self, _X):
        X = np.hstack((np.ones((_X.shape[0], 1)), _X))
        self.PREDICT = []

        for x in X:
            self.PREDICT.append(np.sign(self.W @ x))

        self.PREDICT = np.array(self.PREDICT)

In [197]:
class RegressaoLinear:
    def __init__(self, _X, _Y):
        self.X = np.hstack((np.ones((_X.shape[0], 1)), _X))
        self.Y = _Y
    
    def fit(self):

        matriz_XtX = self.X.T @ self.X
        inversa_matriz_XtX = np.linalg.inv(matriz_XtX)
        matriz_XtY = self.X.T @ self.Y
        
        self.W = inversa_matriz_XtX @ matriz_XtY
    
    def predict(self, _X):
        X = np.hstack((np.ones((_X.shape[0], 1)), _X))
        self.PREDICT = []

        for x in X:
            self.PREDICT.append(np.sign(self.W @ x))

        self.PREDICT = np.array(self.PREDICT)

In [198]:
class RegressaoLogistica:
    def __init__(self, _X, _Y, eta = 0.1, tmax = 1000):
        self.X = np.hstack((np.ones((_X.shape[0], 1)), _X))
        self.Y = _Y
        self.eta = eta
        self.tmax = tmax

    def fit(self):

        self.W = np.zeros(self.X.shape[1])

        for t in range(self.tmax):
            
            somatorio = np.zeros(self.X.shape[1])

            for i in range(self.X.shape[0]):
                exponencial = np.exp(self.Y[i]*(self.W @ self.X[i]))
                somatorio += (self.Y[i]*self.X[i]) / (1 + exponencial)
            
            gradiente = -(1/self.X.shape[0]) * somatorio

            if np.linalg.norm(gradiente) < 0.00000000000001 :
                break

            self.W = self.W - (self.eta * gradiente)

    def predict_prob(self, _X):
        self.PROB = []

        for x in _X:
            z = x @ self.W
            self.PROB.append(1 / (1 + np.exp(-z)))
            
        self.PROB = np.array(self.PROB)

    def predict(self, _X):
        X = np.hstack((np.ones((_X.shape[0], 1)), _X))
        self.predict_prob(X)
        self.PREDICT = np.empty(len(self.PROB), dtype = int)

        for i in range(len(self.PROB)):
            if self.PROB[i] >= 0.5:
                self.PREDICT[i] = 1
            else:
                self.PREDICT[i] = -1